Завантаження даних

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

# Завантаження датасету Fashion MNIST
(train_images, _), (_, _) = fashion_mnist.load_data()

# Нормалізація даних
train_images = (train_images - 127.5) / 127.5

# Розмір зображень
img_shape = train_images.shape[1:]

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Створення генератора

In [5]:
def build_generator():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, input_shape=(100,), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(tf.reduce_prod(img_shape).numpy(), activation='tanh'),
        tf.keras.layers.Reshape(img_shape)
    ])
    return model

Створення дискримінатора

In [6]:
def build_discriminator():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=img_shape),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

Складання моделі GAN

In [7]:
generator = build_generator()
discriminator = build_discriminator()

# Заморожуємо ваги дискримінатора під час тренування генератора
discriminator.trainable = False

gan_input = tf.keras.Input(shape=(100,))
gan_output = discriminator(generator(gan_input))
gan = tf.keras.Model(gan_input, gan_output)

# Компілюємо моделі
generator.compile(loss='binary_crossentropy', optimizer='adam')
gan.compile(loss='binary_crossentropy', optimizer='adam')

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Підготовка даних та тренування моделі

In [ ]:
batch_size = 64
dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(len(train_images)).batch(batch_size)

epochs = 50
for epoch in range(epochs):
    for batch in dataset:
        noise = tf.random.normal(shape=(batch_size, 100))
        
        # Згенерувати зображення
        generated_images = generator(noise)
        
        # Створити неправильні мітки для генератора
        misleading_targets = tf.ones((batch_size, 1))
        
        # Тренування дискримінатора
        expanded_batch = tf.cast(batch, tf.float32)
        combined_images = tf.concat([tf.cast(generated_images, tf.float32), expanded_batch], axis=-1)
        labels = tf.concat([misleading_targets, tf.zeros((batch_size, 1))], axis=0)
        labels += 0.05 * tf.random.uniform(tf.shape(labels))
        
        # Тренування генератора через модель GAN
        noise = tf.random.normal(shape=(batch_size, 100))
        misleading_targets = tf.zeros((batch_size, 1))
        discriminator.trainable = False
        gan.train_on_batch(noise, misleading_targets)

Побудова графіка функції втрат

In [ ]:
import matplotlib.pyplot as plt

def plot_loss(losses):
    plt.figure(figsize=(10, 5))
    plt.plot(losses["generator"], label="generator")
    plt.plot(losses["discriminator"], label="discriminator")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

# Тренувальна петля
def train(generator, discriminator, gan, dataset, epochs, batch_size):
    losses = {"generator": [], "discriminator": []}
    for epoch in range(epochs):
        for batch in dataset:
            noise = tf.random.normal(shape=(batch_size, 100))
            generated_images = generator(noise)
            combined_images = tf.concat([generated_images, tf.expand_dims(batch, axis=-1)], axis=0)
            labels = tf.concat([tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0)
            labels += 0.05 * tf.random.uniform(tf.shape(labels))
            
            discriminator.trainable = True
            d_loss = discriminator.train_on_batch(combined_images, labels)
            
            noise = tf.random.normal(shape=(batch_size, 100))
            misleading_targets = tf.zeros((batch_size, 1))
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, misleading_targets)
        
        losses["generator"].append(g_loss)
        losses["discriminator"].append(d_loss)
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch+1}, Generator Loss: {g_loss}, Discriminator Loss: {d_loss}")
    
    plot_loss(losses)

# Тренування моделі
train(generator, discriminator, gan, dataset, epochs, batch_size)